In [1]:
!pip install tensorboard
!pip install torch 

     |████████████████████████████████| 735.5 MB 2.1 kB/s  eta 0:00:01     |█████▌                          | 126.1 MB 85.0 MB/s eta 0:00:08     |█████████████▌                  | 311.5 MB 62.4 MB/s eta 0:00:07     |█████████████▊                  | 315.2 MB 62.4 MB/s eta 0:00:07


In [2]:
import torch 

In [ ]:
!tensorboard --logdir=checkpoint/


# Webpage Address: https://aaca-qgen-quac.notebook.us-east-1.sagemaker.aws/proxy/6006/#scalars&_smoothingWeight=0


TensorBoard 1.15.0 at http://ip-172-16-9-254:6006/ (Press CTRL+C to quit)
W0325 12:30:51.695362 140098561623808 checkpoint_management.py:288] FailedPreconditionError: checkpoint; Is a directory
W0325 12:30:51.695589 140098561623808 checkpoint_management.py:289] checkpoint/../checkpoint: Checkpoint ignored


In [6]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import json, os, pandas as pd

def extract_metrics(filename):

    model_compare = []

    with open(filename,'r') as f:
        lines = f.readlines()
        #print(lines)

        json_str = str(lines[0]).replace("'", '"')
        
    return json.loads(json_str)

def pull_final_loss(folder):
    
    sub_folds = os.listdir(folder)
    sub_folds.remove('.ipynb_checkpoints')
    
    runs = []
    params = []
    steps = []
    loss = []
    
    for f in sub_folds:
        file = folder + f
        
        event_acc = EventAccumulator(file)
        event_acc.Reload()


        # E. g. get wall clock, number of steps and value for a scalar 'Accuracy'
        w_times, step_nums, vals = zip(*event_acc.Scalars('loss'))
        
        runs.append(f)
        steps.append(step_nums[len(step_nums)-1])
        loss.append(vals[len(vals)-1])
        
        # get hyper param info
        params.append(extract_metrics('results/Hypertuning/QUAC/'+f+'.txt'))

    loss_df = pd.DataFrame(list(zip(runs, params, steps, loss)), columns = ['Run', 'Parameters', 'Steps', 'Loss'])
    
    return loss_df


losses = pull_final_loss('checkpoint/')
losses.to_csv('GPT2_QUAC.csv')